# Explore Simulation with 3 Targets

In [2]:
%load_ext autoreload
%autoreload 2

import higlass as hg
import ipywidgets as widgets
import numpy as np

from utils import create_annotation_track, create_bed_vector_tileset, extract_annotations

experiment = widgets.Dropdown(
    options=[('1.2 Mbps with 100K reads', 'small'), ('12 Mbps with 1M reads', 'large')],
    value='small',
    description='Experiment:',
)

experiment

Dropdown(description='Experiment:', options=(('1.2 Mbps with 100K reads', 'small'), ('12 Mbps with 1M reads', …

In [3]:
if experiment.value == 'small':
    genome_size = int(1.2e6)
    num_reads = int(1e5)
else:
    genome_size = int(1.2e7)
    num_reads = int(1e6)
    
suffix = f"{genome_size}-{num_reads}"

### Tilesets

In [4]:
ts_signal_1 = hg.tilesets.bigwig(f'data/signal-target-1-distorted-peaks-{suffix}.bigWig')
ts_signal_2 = hg.tilesets.bigwig(f'data/signal-target-2-distorted-peaks-{suffix}.bigWig')
ts_signal_3 = hg.tilesets.bigwig(f'data/signal-target-3-distorted-peaks-{suffix}.bigWig')
ts_signal = [ts_signal_1, ts_signal_2, ts_signal_3]

ts_feature_weights = create_bed_vector_tileset(
    f'data/simulated-features-3-targets-{suffix}.bed',
    f'data/simulated-genome-chrom-sizes-{suffix}.tsv',
    uuid = 'feature_weights',
    aggregator = np.max,
    log_scale = True
)

annos_1, annos_2, annos_3, _, _, _, annos_all = extract_annotations(
    f'data/simulated-features-3-targets-{suffix}.bed',
    {
        'BindingA': [0],
        'BindingB': [1],
        'BindingC': [2],
        'BindingAB': [0, 1, 3],
        'BindingAC': [0, 2, 4],
        'BindingBC': [1, 2, 5],
        'BindingABC': [0, 1, 2, 3, 4, 5, 6]
    },
    f'data/simulated-genome-chrom-sizes-{suffix}.tsv',
)

### Tracks

In [6]:
tr_axis = hg.Track(track_type='top-axis', position='top', uid="axis")

tr_annos_1 = create_annotation_track('annos-1', annos_1)
tr_annos_2 = create_annotation_track('annos-2', annos_2)
tr_annos_3 = create_annotation_track('annos-3', annos_3)

tr_feature_weights = hg.Track(
    'horizontal-bar',
    uid='feature-weights',
    tileset=ts_feature_weights,
    position='top',
    height=16,
    options={
        "name": "Feature weights (log)",
        "colorRange": [
            "#f2f2f2", "#f2f2f2", "#f2f2f2", "#f2f2f2", "#dddddd", "#bbbbbb", "#888888"
        ],
        "labelColor": "#666666",
    }
)

tr_signal = []

for i in range(3):
    tr_signal.append(hg.Track(
        'horizontal-bar',
        uid=f"signal-{i + 1}",
        tileset=ts_signal[i],
        position='top',
        height=48,
        options={
            'name': f"Target {i + 1} signal",
            'labelColor': '#6fb2e4',
            'colorRange': ['#6fb2e4', '#0064a8'],
        }
    ))

overlays = [{
    "uid": "overlays",
    "includes": [
        "feature-weights",
        "annos-1",
        "annos-2",
        "annos-3",
        "signal-1",
        "signal-2",
        "signal-3",
    ],
    "options": {
        "extent": [
            [int(x[0]), int(x[1])] for x in annos_all
        ],
        "minWidth": 6,
        "fill": "#c17da5",
        "fillOpacity": 0.1,
        "stroke": "white",
        "strokeWidth": 1,
        "strokePos": ["left", "right"],
        "outline": "#c17da5",
        "outlineWidth": 2,
        "outlinePos": ["left", "right"]
    }
}]

widget, server, _ = hg.display(
    views=[
        hg.View(
            [
                tr_axis,
                tr_feature_weights,
                tr_annos_1,
                tr_annos_2,
                tr_annos_3,
                tr_signal[0],
                tr_signal[1],
                tr_signal[2],
            ],
            uid='v',
            initialXDomain=[0, genome_size],
            overlays=overlays
        )
    ],
    value_scale_syncs=[[
        tr_signal[0],
        tr_signal[1],
        tr_signal[2],
    ]],
    no_fuse=True
)

widget

HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'v', 'tracks': {'top': [{'type': 'top-axis', 'uid…